In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO, emit
from os import environ as env

In [ ]:
def run_display():
    app = Flask('display', static_url_path='/', static_folder='../display')
    socketio = SocketIO(app)
    
    state = {
        'resolution': None
    }

    @app.route("/display")
    def home():
        return app.send_static_file("display.html")

    @socketio.on('handshake', namespace='/display')
    def handle_handshake_display():
        print('client connected [display]')
        return 'ack'
    
    @socketio.on('handshake', namespace='/control')
    def handle_handshake_control():
        print('client connected [control]', state)
        emit('resolution', state['resolution'], namespace='/control', broadcast=False)
        return 'ack'

    @socketio.on('resolution', namespace='/display')
    def handle_resolution_display(w, h):
        print('old resolution:', state['resolution'])
        state['resolution'] = (w, h)
        print('new resolution:', state['resolution'])
        emit('resolution', state['resolution'], namespace='/control', broadcast=True)

    @socketio.on('highlight', namespace='/control')
    def handle_highlight(*pos):
        print('broadcasting highlight:', pos)
        emit('hightlight', pos, namespace='/display', broadcast=True)
        return 'broadcasted'

    @socketio.on('show_image', namespace='/control')
    def handle_show_image(*args):
        print('broadcasting show_image')
        emit('show_image', args, namespace='/display', broadcast=True)
        return 'broadcasted'

    hostname = env['HOST'].strip()
    print('now go to http://' + hostname + ':5000/display?res_x=20&res_y=20')
    socketio.run(app, host=hostname)

In [ ]:
run_display()